In [2]:
import os
from PIL import Image
import numpy as np

In [10]:
def get_average_image_size(dataset_path):
    widths, heights = [], []
    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                try:
                    img_path = os.path.join(class_path, img_name)
                    with Image.open(img_path) as img:
                        w, h = img.size
                        widths.append(w)
                        heights.append(h)
                except:
                    continue
    avg_width = int(np.mean(widths))
    avg_height = int(np.mean(heights))
    return avg_width, avg_height

def load_caltech20_dataset(dataset_path, resize=False):
    avg_w, avg_h = get_average_image_size(dataset_path)
    data, labels = [], []
    classes = sorted(os.listdir(dataset_path))

    for label, class_name in enumerate(classes):
        class_path = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                try:
                    img = Image.open(img_path).convert("RGB")
                    if resize:
                        img = img.resize((avg_w, avg_h))
                    data.append(np.array(img))
                    labels.append(label)
                except:
                    print(f"Erreur avec {img_path}")

    return np.array(data), np.array(labels), classes

In [11]:
X, y, class_names = load_caltech20_dataset("caltech20")
print(X.shape, y.shape)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (1051,) + inhomogeneous part.

In [12]:
print(f"Nombre d'images : {len(X)}")
print(f"Nombre de classes : {len(class_names)}")
print(f"Exemples de classes : {class_names[:5]}")

Nombre d'images : 1051
Nombre de classes : 20
Exemples de classes : ['ant', 'beaver', 'brontosaurus', 'cannon', 'chair']


In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")

Training set size: 840
Testing set size: 211


pip install opencv-contrib-python scikit-learn
pip install opencv-python

In [17]:
import cv2
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

def extract_sift_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return descriptors

def load_descriptors(image_paths):
    all_desc = []
    for path in image_paths:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        desc = extract_sift_features(img)
        if desc is not None:
            all_desc.append(desc)
    return all_desc

def sample_descriptors(all_descriptors, max_samples=10000):
    desc_stack = np.vstack(all_descriptors)
    indices = np.random.choice(len(desc_stack), min(max_samples, len(desc_stack)), replace=False)
    return desc_stack[indices]

def reduce_dimensionality(descriptors, n_components=64):
    pca = PCA(n_components=n_components)
    return pca.fit_transform(descriptors), pca

def build_codebook(descriptors, k=100):
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(descriptors)
    return kmeans

def compute_bow_histogram(image, sift, pca, kmeans):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, desc = sift.detectAndCompute(gray, None)
    if desc is None:
        return np.zeros(kmeans.n_clusters)
    desc = pca.transform(desc)
    labels = kmeans.predict(desc)
    hist, _ = np.histogram(labels, bins=np.arange(kmeans.n_clusters + 1))
    return hist / np.linalg.norm(hist)  # Normalize

def encode_dataset(image_paths, sift, pca, kmeans):
    features = []
    for path in image_paths:
        img = cv2.imread(path)
        hist = compute_bow_histogram(img, sift, pca, kmeans)
        features.append(hist)
    return np.array(features)


ImportError: libGL.so.1: cannot open shared object file: No such file or directory